# Laboratório 1
*Projeto Aprendizado de Máquina*

*Aluno: Vinícius José de Menezes Pereira*

## Import libraries

In [ ]:
import pandas as pd
import numpy as np

import random
import heapq

from pprint import pprint

## Read datasets

In [ ]:
movies = pd.read_csv('movies.csv', encoding='utf-8', sep=";", error_bad_lines=False) 
ratings = pd.read_csv("ratings.csv",encoding='utf-8', sep=";", error_bad_lines=False) 
users = pd.read_csv("users.csv",encoding='utf-8', sep=",", error_bad_lines=False) 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 3777: expected 3 fields, saw 4\n'
b'Skipping line 537689: expected 4 fields, saw 7\nSkipping line 634038: expected 4 fields, saw 7\n'
b'Skipping line 778632: expected 4 fields, saw 7\n'
b'Skipping line 923172: expected 4 fields, saw 7\n'
b'Skipping line 1164230: expected 4 fields, saw 5\n'
b'Skipping line 1212461: expected 4 fields, saw 6\nSkipping line 1308907: expected 4 fields, saw 6\n'
b'Skipping line 1453497: expected 4 fields, saw 6\nSkipping line 1549931: expected 4 fields, saw 6\n'


## Data Preparation

### Data cleaning

- Nesta parte, percebeu-se que não haviam dados faltantes, não sendo necessário completá-los.

#### Users

In [ ]:
print(users.shape)
users = users.dropna(how = 'any')
print(users.shape)

(6040, 6)
(6040, 6)


#### Movies


In [ ]:
print(movies.shape)
movies = movies.dropna(how = 'any')
print(movies.shape)

(3882, 3)
(3882, 3)


#### Ratings

In [ ]:
print(ratings["Rating"].min())
print(ratings["Rating"].max())
print(ratings.shape)
ratings = ratings.dropna(how = 'any')
print(ratings.shape)

1.0
5.0
(1848212, 4)
(1848204, 4)


### Data transformation

#### Users

Transformações realizadas:
- Determinar ano de nascimento
- Discretização da idade
- Passar o atributo gênero para variável numérica

In [ ]:
# Setting birth Year and Age 
users = pd.concat([users, users["birthday"].str.split(pat='/',expand = True)], axis=1)
users = users.drop(0,axis = 1)
users = users.drop(1,axis = 1)
users.rename(columns = {2:'Year'}, inplace = True)
users["Year"] = pd.to_numeric(users["Year"])
users["Age"] = 2022-users["Year"]
users = users.drop('Year',axis = 1)

In [ ]:
# Age Discretization
users["Age"].loc[users["Age"] < 18] = 1
users["Age"].loc[(users["Age"] > 17)&(users["Age"] < 18)] = 18
users["Age"].loc[(users["Age"] > 24)&(users["Age"] < 35)] = 25
users["Age"].loc[(users["Age"] > 34)&(users["Age"] < 45)] = 35
users["Age"].loc[(users["Age"] > 44)&(users["Age"] < 50)] = 45
users["Age"].loc[(users["Age"] > 49)&(users["Age"] < 56)] = 50
users["Age"].loc[users["Age"] > 55] = 56

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# Gender 
users["Gender"].loc[users["Gender"] == 'M'] = 1
users["Gender"].loc[users["Gender"] =='F'] = 0
users["Gender"] = pd.to_numeric(users["Gender"])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Data reduction

#### Users
- Coluna nome irrelevante
- Por simplicidade, não foi considerado o *Zip-Code*, até por que seria trabalhoso identificar a localização
- *Birthday* redundate, já que já temos *Age*

In [ ]:
users = users.drop('name', axis = 1)
users = users.drop('Zip-code', axis = 1)
users = users.drop('birthday', axis = 1)

In [ ]:
users.head()

,UserID,Gender,Occupation,Age
0,1,0,10,1
1,2,1,16,56
2,3,1,15,25
3,4,1,7,45
4,5,1,20,25


#### Movies

- Por simplicidade, a única característica do filme utilizada será seu número de identificação
- Verificou-se também que a separação dos gêneros em cada linhas deixava a *behavior tree* muito grande

In [ ]:
movies = movies.drop("Genres",axis = 1)
movies = movies.drop('Title',axis = 1)

In [ ]:
movies.head()

,MovieID
0,1
1,2
2,3
3,4
4,5


#### Ratings

- Coluna *Timestamp* irrelevante neste trabalho

In [ ]:
ratings = ratings.drop('Timestamp', axis = 1)

In [ ]:
ratings.head()

,UserID,MovieID,Rating
0,1,1193.0,5.0
1,1,661.0,3.0
2,1,914.0,3.0
3,1,3408.0,4.0
4,1,2355.0,5.0


### Joing datasets

- *UserID* necessário apenas como uma chave para "imergir" os dados
- Não deseja-se utilizar o *UserID* no algoritmo de aprendizado
- Colocou-se a coluna alvo *Rating* na última posição das colunas


In [ ]:
df = pd.merge(ratings,users)
df = pd.merge(df,movies)
df = df.drop('UserID',axis = 1)
label = df['Rating']
df = df.drop('Rating',axis = 1)
df = pd.concat([df,label],axis=1)

In [ ]:
df.head()

,MovieID,Gender,Occupation,Age,Rating
0,1193.0,0,10,1,5.0
1,1193.0,1,16,56,5.0
2,1193.0,1,12,25,4.0
3,1193.0,1,7,25,4.0
4,1193.0,1,1,50,5.0


## Spliting datatset

Divisão dos dados em três conjuntos:
 - Train
 - Validation
 - Test

In [ ]:
def split(df,validation_size,test_size):
    #test df
    test_size = round(test_size*len(df))
    indices = df.index.tolist()
    test_indices = random.sample(population = indices, k = test_size)
    test_df = df.loc[test_indices]
    #validation df
    train_df = df.drop(test_indices)
    validation_size = round(validation_size*len(df))
    indices = train_df.index.tolist()
    validation_indices = random.sample(population = indices, k = validation_size)
    validation_df = train_df.loc[validation_indices]
    #train df
    train_df = train_df.drop(validation_indices)
    
    return train_df,validation_df,test_df

In [ ]:
train,validation,test = split(df,0.2,0.2)

## Decision Tree

Implementação da árvore de decisão ID3, em que deseja-se pouca profundidade e escolher o nós baseados no melhor ganho de informação.

### Helper functions
- entropy(data): dado uma *data*, calcula a entropia da coluna alvo e retorna-a.
- info_gain(data,feature_name): determina o ganho de informação de um conjunto de dados ao serará-lo com base numa *feature*. Retorna-se o negativo do ganho e o nome da *feature*.
- choose_best_feature(data): dado uma *data*, analisa suas *features* e retorna a feature de maior ganho. Para determinar a melhor feature, utiliza a estrutura de dados *priority_queue* constrída por *heap* e ordenando pela variável *-gain* e retira-se o último elemento.
- majority(data,p): heurística que permita retorno atencipado de média caso um porcentagem p dos valores do *label* tenha o mesmo valor.

In [ ]:
def entropy(data):
    label = data.columns[-1]
    _,counts = np.unique(data[label], return_counts = True)
    prob = counts/(counts.sum())
    entropy = sum(-prob*np.log2(prob))
    return entropy

In [ ]:
def info_gain(data,feature_name):
    feature_value,counts = np.unique(data[feature_name], return_counts = True)
    lower_entropy = 0
    for i in range(len(feature_value)):
        filter = data.loc[data[feature_name] == feature_value[i]]
        lower_entropy += counts[i]*entropy(filter)/sum(counts)
    upper_entropy = entropy(data)
    gain = upper_entropy-lower_entropy
    return -gain,feature_name

In [ ]:
def choose_best_feature(data):
    pq = []
    label = data.columns[-1]
    data = data.drop(label,axis=1)
    features = data.columns
    for feature in features:
        heapq.heappush(pq, info_gain(data,feature))
    _,best_feature = heapq.heappop(pq)
    return best_feature

In [ ]:
def majority(data,p):
    label = data.columns[-1]
    value,counts = np.unique(data[label],return_counts=True)
    counts = counts/sum(counts)
    max = np.max(counts)
    if max > p:
        return True
    else: return False

### Main algorithm

ID3(data,depth,parant_node=None):
- Árvore constrída em estrutura de dicionário.
- depth é o máximo valor da profundidade da árvore.
- parant_node é utilizado para retornar o melhor atributo, ou seja, nó anterior, caso o filtro do atributo não possua exemplos no *label*
- Primeiramente a raiz da árvore é o melhor atributo
- Para cada valor do melhor atributo, cria-se uma ramo da árvore, filtrando a *data* e e retirando o melhor atritubuto
- Chama-se recursivamente ID3 em cada ramo
- Por fim adiciona-se o ramo na árvore


In [ ]:
def ID3(data,depth,parent_node = None):
    label = data.columns[-1]
    use_majority = majority(data,1)
    if len(data[label]) == 0:
        return parent_node
    elif use_majority or len(data.columns)-1 == 0 or depth == 0:
        return round(data[label].mean())
    else:
        best_feature = choose_best_feature(data)
        root = {best_feature:{}}
        best_values = np.unique(data[best_feature])
        for value in best_values:
            branch_data = data.loc[data[best_feature] == value]
            branch_data = branch_data.drop(best_feature,axis=1)
            branch = ID3(branch_data,depth-1,best_feature)
            root[best_feature][value] = branch
    return root

In [ ]:
tree = ID3(train,4,None)
pprint(tree)

A saída de streaming foi truncada nas últimas 5000 linhas.
                                                             3420.0: 3,
                                                             3421.0: 1,
                                                             3426.0: 4,
                                                             3427.0: 3,
                                                             3430.0: 4,
                                                             3431.0: 1,
                                                             3432.0: 1,
                                                             3433.0: 1,
                                                             3435.0: 4,
                                                             3441.0: 2,
                                                             3448.0: 4,
                                                             3452.0: 3,
                                                             3462.0: 4,
     

### Test tree
- predict(query,tree): *query* é uma linha dicionarizada. O *predict* irá relacionar a tree e a *query* por suas chaves e irá retornar a previsão da linha conforme a *tree*.
- accuracy(test,tree): calcula a acurácia da ID3 no conjunto teste e retorna o conjunto teste com a classificação.
- confusion_matrix(data,classification,rating): retorna matrix de confusão comparando classificação e rating.
- stats(matrix): baseado na matriz de confusão, devolve estatísticas importantes.
- error(data,rating,classification): retorna error médio quadrádico de uma classificação dado um rating
- kappa(actual,random): calcular quão bom é um classificador

In [ ]:
def predict(query,tree):
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return 4
            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
        


In [200]:
def accuracy(test,tree,Rating):
    queries = test.iloc[:,:-1].to_dict(orient="records")
    predicts = []
    test['classification']= 0*test[Rating]
    test['correct'] = 0*test[Rating]
    for i in range(len(test.index)):
      predicts.append(predict(queries[i],tree))
    test['classification'] = predicts
    test['correct'].loc[test['classification'] == test[Rating]] = 1
    accuracy = test['correct'].mean()
    return accuracy,test
    

In [ ]:
ac,result = accuracy(test,tree)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [257]:
print(ac)


0.4517409301193085


In [175]:
def confusion_matrix(data,classification,Rating):
  matrix = np.zeros((5, 5))
  for i in data.index:
      u = data[classification].loc[i]
      v = data[Rating].loc[i]
      matrix[int(v)-1][int(u)-1] += 1.0
  return matrix

        

In [ ]:
def stats(matrix):
  tot = 0
  precision = np.zeros(5)
  precision_sum = np.zeros(5)
  recall = np.zeros(5)
  recall_sum = np.zeros(5)
  success = 0
  for i in range(5):
    for j in range(5):
      if i == j:
        success += matrix[i][j]
        precision[i] = matrix[i][j]
        recall[i] = matrix[i][j]
      precision_sum[i] += matrix[i][j]
      recall_sum[j] += matrix[i][j]
      tot += matrix[i][j]

  success = success/tot
  precision = precision/precision_sum
  recall = recall/recall_sum
  return success,precision,recall

In [ ]:
matrix = confusion_matrix(result.iloc[1:100])
print(matrix)
success,precision,recall = stats(matrix)
print(success,precision,recall)

[[ 1.  3.  1.  1.  0.]
 [ 0.  3.  3.  3.  0.]
 [ 0.  3. 10. 10.  0.]
 [ 0.  2.  6. 25.  0.]
 [ 0.  0.  3. 13. 12.]]
0.5151515151515151 [0.16666667 0.33333333 0.43478261 0.75757576 0.42857143] [1.         0.27272727 0.43478261 0.48076923 1.        ]


In [266]:
def error(data,rating,classification):
  data['error'] = (data[rating]-data[classification])**2
  error = np.sqrt(data['error'].mean())/np.sqrt(len(data.index))
  return error

In [270]:
def kappa(actual,random):
  k = actual-random/(1-random)
  return k

## A priori classifier

O classificador a priori classifica o filme conforme a média dos usuários

In [275]:
ratings['MovieID1'] = ratings['MovieID']
priori = ratings.groupby(ratings['MovieID1']).mean()
priori['priori'] = round(priori['Rating'])
priori = priori.drop('UserID',axis=1)
priori = priori.drop('Rating',axis=1)


## Analisys

Cria-se uma tabela com meus ratings e compara-se os classificadores a priori e da árvore de decisão.

In [258]:
my = pd.read_csv('my.csv', encoding='utf-8', sep=",", error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [259]:
r,table = accuracy(my,tree,'My_Rating')
table = pd.merge(table,priori)
table.head(10)

,MovieID,Gender,Occupation,Age,My_Rating,classification,correct,priori
0,1,1,4,18,3,4,0,4.0
1,48,1,4,18,4,4,1,3.0
2,364,1,4,18,5,4,0,4.0
3,588,1,4,18,5,4,0,4.0
4,595,1,4,18,4,4,1,4.0
5,1032,1,4,18,4,4,1,4.0
6,1566,1,4,18,4,5,0,3.0
7,1907,1,4,18,5,4,0,4.0
8,2085,1,4,18,2,4,0,4.0
9,2088,1,4,18,3,3,1,3.0


In [274]:
matrix_priori = confusion_matrix(table,'priori','My_Rating')
print(stats(matrix_priori))
print(matrix_priori)
priori_error = error(table,'My_Rating','priori')
print(priori_error)
priori_kappa = kappa(stats(matrix_priori)[0],ac)
print(priori_kappa)

(0.3, array([nan, 0. , 0.5, 0.5, 0. ]), array([       nan,        nan, 0.33333333, 0.28571429,        nan]))
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 2. 2. 0.]
 [0. 0. 0. 3. 0.]]
0.31622776601683794
-0.5239552338233335


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [273]:
matrix_tree = confusion_matrix(table,'classification','My_Rating')
print(stats(matrix_tree))
print(matrix_tree)
tree_error = error(table,'My_Rating','classification')
print(tree_error)
tree_kappa = kappa(stats(matrix_tree)[0],ac)
print(tree_kappa)

(0.4, array([ nan, 0.  , 0.5 , 0.75, 0.  ]), array([  nan,   nan, 1.   , 0.375, 0.   ]))
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 1. 0.]
 [0. 0. 0. 3. 1.]
 [0. 0. 0. 3. 0.]]
0.3
-0.42395523382333344


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


## Resultados

Pelas três estatísticas utilizadas para avaliar os modelos de classificação utilizados, vemos que a decision tree apresentou um melhor desempenho, mas não tão melhor, pois estamos usando um pequeno dataset para avaliar. Caso o dataset fosse maior, veríamos uma maior discrepância. Equanto a acurácia de a árvore é 0.4, a do classificador a priori é 0.3. Já o erro da árvore é 0.3 e o erro do classificador a priori é 0.31. As matrizes de confusão se parecem, devido ao pequeno número de dados.

O treinamento a árvore ID3 demorou bastante, pois classificava por filmes, e há muitos filmes. A melhor feature demonstrada foi a idade, seguida pela ocupação, gênero e filme. A acurácia atingida foi de 0.42 para o conjunto de dados de teste. Caso não considerássimos como nós dar árvore o parâmetro MovieID, a acurácia caia para cerca de 0.33. Não foi necessário utilizar dados de validação, pois não havia muito escolho para os parâmetros. Os parâmetros estavam como que já determinados, pois adicionar outros parâmetros como gênero do filme, o que foi feito, dificultava muito o problema, pois a árvore ficava muito larga e profunda e não ganhava considerável aumento de precisão.

## Conclusão

Achei o trabalho muito bom, realmente consegui aprender bastante, e fico muito feliz por isso. Mas também achei ele muito difícil, gastei realmente muitas horas fazendo-o, pois fiz-o sozinho, já que ninguém se ofereceu para fazer dupla. Confesso que o fim do código está menos bem feito, pois fiz-o bem cançado.

## Descrição da implementação

Implementação realizada em python na IDE google colab. A célula de treinamento demora bastante, por volta de 13 min. O resto é rodado de forma rápida. Necessário é dar upload nos arquivos para rodar.